# Setup

In [16]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.5f}'.format}, suppress = True)

from scipy.interpolate import RegularGridInterpolator

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook

MAGFIELDFILENAME1 = "magfields/bfield_old.txt"
MAGFIELDFILENAME2 = "magfields/bfield.txt"

In [17]:
# For some reason this commands needs to be read twice to actually work
%matplotlib notebook

# File reading

In [18]:
def unpack_magfield_file(filename):
    B = [[], [], [], [], [], []]
    f = open(filename, "r")
    
    for line in f:
        mag_meas = line.replace("\n", "").split(' ')
        for i in range(len(mag_meas)):
            mag_meas[i] = float(mag_meas[i])
        if mag_meas[0] not in B[0]:
            B[0].append(mag_meas[0])
        if mag_meas[1] not in B[1]:
            B[1].append(mag_meas[1])
        if mag_meas[2] not in B[2]:
            B[2].append(mag_meas[2])
        B[3].append(mag_meas[3])
        B[4].append(mag_meas[4])
        B[5].append(mag_meas[5])

    f.close()
    
    return B

def pack_data(orig_arr, dest_size):
    dest_arr = []
    it       = 0
    for d in orig_arr:
        if it == 0:
            dest_arr.append([d])
        else:
            dest_arr[-1].append(d)
        it += 1
        if it == dest_size:
            it = 0
    return dest_arr

# Magnetic Field Estimation

In [19]:
def create_interpolators(x, y, z, bx, by, bz):
    bx_z = pack_data(bx, len(z))
    by_z = pack_data(by, len(z))
    bz_z = pack_data(bz, len(z))

    bx_y = np.array(pack_data(bx_z, len(y)))
    by_y = np.array(pack_data(by_z, len(y)))
    bz_y = np.array(pack_data(bz_z, len(y)))

    bx_interpolator = RegularGridInterpolator((x, y, z), bx_y, method="linear", bounds_error=True)
    by_interpolator = RegularGridInterpolator((x, y, z), by_y, method="linear", bounds_error=True)
    bz_interpolator = RegularGridInterpolator((x, y, z), bz_y, method="linear", bounds_error=True)
    
    return (bx_interpolator, by_interpolator, bz_interpolator)

def get_b(l, B):
    return [B[0]([l[0], l[1], l[2]])[0], B[1]([l[0], l[1], l[2]])[0], B[2]([l[0], l[1], l[2]])[0]]

# Test

In [21]:
d  = unpack_magfield_file(MAGFIELDFILENAME1)
B1 = create_interpolators(d[0], d[1], d[2], d[3], d[4], d[5])

d  = unpack_magfield_file(MAGFIELDFILENAME2)
B2 = create_interpolators(d[0], d[1], d[2], d[3], d[4], d[5])

print(get_b((42, 37, 302), B1))
print(get_b((42, 37, 302), B2))

[0.11528114289320006, -0.38206919875000017, -0.1751713338111999]
[0.11529412000000001, -0.38209438400000006, -0.175190608]
